<a href="https://colab.research.google.com/github/allydrzewo/CS370-Assignments/blob/main/assignment-2/Assignment2A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import matplotlib.pyplot as plt
from captum.attr import IntegratedGradients, LayerGradCam
from captum.attr import visualization as viz

# Define directories
base_dir = 'dogscats/subset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Data transformations (resize, normalize, etc.)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, data_transforms['train'])
val_dataset = datasets.ImageFolder(validation_dir, data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, data_transforms['val'])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # For testing, use batch size 1

# Load a pretrained model
model = models.resnet18(pretrained=True)

# Modify the final layer to fit 2-class problem (cats vs dogs)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes, cat & dog

# Set model to evaluation mode
model.eval()

# Sample input
sample_input, _ = next(iter(test_loader))

# Move sample input to the correct device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
sample_input = sample_input.to(device)

# Perform a forward pass to get the model's predictions
output = model(sample_input)
predicted_class = torch.argmax(output).item()

# Normalize and prepare the input for attribution methods
original_image = sample_input.squeeze().cpu().detach().numpy().transpose(1, 2, 0)
original_image = original_image * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]  # Unnormalize
original_image = np.clip(original_image, 0, 1)

FileNotFoundError: [Errno 2] No such file or directory: 'dogscats/subset/train'

In [ ]:
# Integrated Gradients
ig = IntegratedGradients(model)

# Calculate integrated gradients for predicted class
attributions_ig = ig.attribute(sample_input, target=predicted_class, n_steps=50)

# Visualize the attributions using Captum visualization tools
_ = viz.visualize_image_attr(
    attributions_ig.squeeze().cpu().permute(1, 2, 0).detach().numpy(),
    original_image,
    method="heat_map",
    sign="all",
    show_colorbar=True,
    title="Integrated Gradients Attributions"
)

In [ ]:
# Grad-CAM
layer_gc = LayerGradCam(model, model.layer4[-1])  # Change layer based on architecture

# Calculate Grad-CAM attributions
attributions_gc = layer_gc.attribute(sample_input, target=predicted_class)

# Upsample the attribution map to the input image size
attributions_gc_upsampled = F.interpolate(attributions_gc, sample_input.shape[2:], mode="bilinear")

# Visualize Grad-CAM
_ = viz.visualize_image_attr(
    attributions_gc_upsampled.squeeze().cpu().permute(1, 2, 0).detach().numpy(),
    original_image,
    method="heat_map",
    sign="all",
    show_colorbar=True,
    title="Grad-CAM"
)